# Set up Model Predictive Control Using Do-MPC Package

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import casadi as cas
import do_mpc

from cas_models.continuous_time.models import StateSpaceModelCT
from cas_models.transformations import connect_systems
from cas_models.discrete_time.models import StateSpaceModelDTFromCT
from cas_models.discrete_time.simulate import (
    make_n_step_simulation_function_from_model,
)

from feed_conc_ctrl.models import MixingTankModelCT, FlowMixerCT
from feed_conc_ctrl.plot_utils import make_tsplots

## Example: Four-Tank Feed Blending System

```none
                      ┌────────┐   
                   ┌─►┤ Tank 2 │   
                   │  │        │   
                   │  │        ├──┐ 
                   │  └────────┘  │ 
     ┌────────┐    │              │     ┌────────┐
────►┤ Tank 1 │    │              ├────►┤ Tank 4 │
     │        │    │              │     │        │
     │        ├────┤  ┌────────┐  │     │        ├────►
     └────────┘    └─►┤ Tank 3 │  │     └────────┘
                      │        │  │ 
                      │        ├──┘ 
                      └────────┘    
```

States
 1. Tank 1 level
 2. Tank 1 mass
 3. Tank 2 level
 4. Tank 2 mass
 5. Tank 3 level
 6. Tank 3 mass
 7. Tank 4 level
 8. Tank 4 mass

Inputs:
 1. Tank 1 inflow rate
 2. Tank 1 inflow concentration
 3. Tank 1 outflow rate
 4. Tank 2 inflow rate
 5. Tank 2 outflow rate
 6. Tank 3 inflow rate
 7. Tank 3 outflow rate
 8. Tank 4 outflow rate

Outputs:
 1. Tank 1 level
 2. Tank 1 mass
 3. Tank 1 outflow concentration
 4. Tank 2 level
 5. Tank 2 mass
 6. Tank 2 outflow concentration
 7. Tank 3 level
 8. Tank 3 mass
 9. Tank 3 outflow concentration
10. Tank 4 level
11. Tank 4 mass
12. Tank 4 outflow concentration
13. Mixer outflow rate
14. Mixer outflow concentration

In [ ]:
results_dir = Path("results")
plot_dir = results_dir / "plots"
results_dir.mkdir(exist_ok=True)
plot_dir.mkdir(exist_ok=True)

In [ ]:
def print_sys_dimensions(sys):
    print(sys.name, f"({sys.ny}x{sys.nu})")
    for attr_name in ["input_names", "state_names", "output_names"]:
        print(f"{attr_name:>15s}: {getattr(sys, attr_name)}")

## Construct Mixing Tank System Model using Casadi-Models Package

In [ ]:
D = 3  # tank diameter [m]
n_tanks = 4
tank_names = [f"tank_{i + 1}" for i in range(n_tanks)]

# Initialize tank system models
systems = [MixingTankModelCT(D=D, name=name) for name in tank_names]

# Add a flow mixer to join flows from two tanks
systems.insert(3, FlowMixerCT(2, name="mixer"))

# Check inputs and outputs
for sys in systems:
    print_sys_dimensions(sys)

In [ ]:
# Connect all systems together
connections = {
    "tank_1_v_dot_out": ["tank_2_v_dot_in", "tank_3_v_dot_in"],
    "tank_2_v_dot_out": "mixer_v_dot_in_1",
    "tank_3_v_dot_out": "mixer_v_dot_in_2",
    "tank_4_v_dot_in": "mixer_v_dot_out",
    "tank_2_conc_in": "tank_1_conc_out",
    "tank_3_conc_in": "tank_1_conc_out",
    "mixer_conc_in_1": "tank_2_conc_out",
    "mixer_conc_in_2": "tank_3_conc_out",
    "tank_4_conc_in": "mixer_conc_out",
}

model_class = StateSpaceModelCT
feed_tanks_system = connect_systems(
    systems,
    connections,
    model_class,
    name="tank_system_21",
    verbose_names=True,
)
print_sys_dimensions(feed_tanks_system)

## Control Model Design

**States**
 1. Tank 1 level
 2. Tank 1 mass
 3. Tank 2 level
 4. Tank 2 mass
 5. Tank 3 level
 6. Tank 3 mass
 7. Tank 4 level
 8. Tank 4 mass

**Manipulated Variables (MVs)**
 1. Tank 2 inflow rate
 2. Tank 3 inflow rate
 3. Mixer inflow 1 rate
 4. Mixer inflow 2 rate
 5. Tank 4 outflow rate

**Unmeasured Disturbances**
 1. Tank 1 inflow rate
 2. Tank 1 inflow concentration

**Controlled Variables (CVs)**
 1. Tank 1 level
 2. Tank 1 outflow concentration
 3. Tank 2 level
 4. Tank 2 outflow concentration
 5. Tank 3 level
 6. Tank 3 outflow concentration
 7. Tank 4 level
 8. Tank 4 outflow concentration



## Construct Do-MPC Model from Casadi System Model

Do MPC model variable types

  | Long name | Short name | Remark   |
  |-----------|------------|----------|
  | states    | `_x`       | Required |
  | inputs    | `_u`       | Required |
  | algebraic | `_z`       | Optional |
  | parameter | `_p`       |          |

In [ ]:
feed_tanks_system.output_names

In [ ]:
control_design = {
    "system_states": [
        "tank_1_L",
        "tank_1_m",
        "tank_2_L",
        "tank_2_m",
        "tank_3_L",
        "tank_3_m",
        "tank_4_L",
        "tank_4_m",
    ],
    "manipulated_variables": [
        "tank_2_v_dot_in",
        "tank_3_v_dot_in",
        "mixer_v_dot_in_1",
        "mixer_v_dot_in_2",
        "tank_4_v_dot_out",
    ],
    "unmeasured_disturbances": [
        "tank_1_v_dot_in",
        "tank_1_conc_in",
    ],
    "controlled_variables": [
        "tank_1_L",
        "tank_1_conc_out",
        "tank_2_L",
        "tank_2_conc_out",
        "tank_3_L",
        "tank_3_conc_out",
        "tank_4_L",
        "tank_4_conc_out",
    ],
}

mpc_params = {
    "n_horizon": 50,  # Prediction horizon (hours)
    "t_step": 1.0,  # Time step (hours)
    "n_robust": 0,  # No robust horizon for now
    "store_full_solution": True,
}

cv_weights = {
    "tank_1_L": 0.1,
    "tank_4_v_dot_out": 1.0,  # Note: this is an MV
    "tank_4_conc_out": 100.0,  # Most important
}

setpoints = {
    "tank_1_L": 1.0,
    "tank_4_v_dot_out": 1.0,  # Note: this is an MV
    "tank_4_conc_out": 2.0,
}

mv_weights = {
    "tank_2_v_dot_in": 0.1,
    "tank_3_v_dot_in": 0.1,
    "mixer_v_dot_in_1": 0.1,
    "mixer_v_dot_in_2": 0.1,
    "tank_4_v_dot_out": 10.0,
}

bounds = {
    "inputs": {
        "tank_2_v_dot_in": {"lower": 0.0, "upper": 2.0},
        "tank_3_v_dot_in": {"lower": 0.0, "upper": 2.0},
        "mixer_v_dot_in_1": {"lower": 0.0, "upper": 2.0},
        "mixer_v_dot_in_2": {"lower": 0.0, "upper": 2.0},
        "tank_4_v_dot_out": {"lower": 0.0, "upper": 2.0},
    },
    "system_states": {
        "tank_1_L": {"lower": 0.1, "upper": 3.0},
        "tank_2_L": {"lower": 0.1, "upper": 3.0},
        "tank_3_L": {"lower": 0.1, "upper": 3.0},
        "tank_4_L": {"lower": 0.1, "upper": 3.0},
    },
    "outputs": {
        "tank_4_conc_out": {"lower": 0.0, "upper": 3.0},
    },
}

In [ ]:
from do_mpc_utils.mpc_constructor import construct_mpc

system = feed_tanks_system

mpc, model = construct_mpc(
    system,
    control_design,
    mpc_params,
    cv_weights,
    setpoints=setpoints,
    mv_weights=mv_weights,
    bounds=bounds,
)

## Simulate MPC with Full State Measurement

In [ ]:
def generate_random_steps_beta(
    nT,
    step_length,
    y_min=0.0,
    y_max=1.0,
    a=10,
    b=10,
    y_range=(0.5, 3.5),
    rng=None,
    seed=10,
):
    if rng is None:
        rng = np.random.default_rng(seed)
    n_steps = nT // step_length
    step_levels = y_min + (y_max - y_min) * rng.beta(a, b, size=n_steps)
    step_sequence = np.repeat(step_levels, step_length)[:nT]
    return step_sequence

In [ ]:
# ========================================
# 4. Setup Simulator (Perfect Model)
# ========================================
simulator = do_mpc.simulator.Simulator(model)
simulator.set_param(t_step=1.0)  # Same as MPC
simulator.setup()


# ========================================
# 5. Setup Estimator (Perfect State Feedback for Testing)
# ========================================
# This only works when outputs are same as states
# estimator = do_mpc.estimator.StateFeedback(model)
# Note: no need for estimator.setup()

# ========================================
# 6. Set Initial Conditions
# ========================================

# Initial state - order must match model.x.keys()
x0_init = {
    # System states
    "tank_1_L": 1.0,
    "tank_1_m": np.pi * (D / 2) ** 2 * 2.0 * 1.0,
    "tank_2_L": 1.0,
    "tank_2_m": np.pi * (D / 2) ** 2 * 2.0 * 1.0,
    "tank_3_L": 1.0,
    "tank_3_m": np.pi * (D / 2) ** 2 * 2.0 * 1.0,
    "tank_4_L": 1.0,
    "tank_4_m": np.pi * (D / 2) ** 2 * 2.0 * 1.0,
    # Disturbance states (true values for simulation)
    "tank_1_v_dot_in": 1.0,
    "tank_1_conc_in": 2.0,
}
assert list(x0_init.keys()) == model.x.keys()
x0_init = np.array(list(x0_init.values())).reshape(-1, 1)

# Set initial state for all components
mpc.x0 = x0_init
simulator.x0 = x0_init

# Calculate initial outputs
# Assumes no direct transmission from u(0) to y(0).
y0_init = simulator.model._meas_fun(
    simulator.x0,  # _x: states
    np.zeros((simulator.model.n_u, 1)),  # _u: inputs
    np.zeros((simulator.model.n_z, 1)),  # _z: algebraic variables
    np.zeros((simulator.model.n_tvp, 1)),  # _tvp: time-varying parameters
    np.zeros((simulator.model.n_p, 1)),  # _p: parameters
    np.zeros((simulator.model.n_y, 1)),  # _v: measurement noise
)

# Set initial guess for MPC
mpc.set_initial_guess()

n_steps = 100  # Simulate 100 hours

# Disturbance inputs:
tank_1_conc_in = generate_random_steps_beta(n_steps, 20, 0.5, 3.5, seed=10)
dist_inputs = np.zeros((n_steps, 2))
dist_inputs[:, 0] = 1.0  # tank_1_v_dot_in
dist_inputs[:, 1] = tank_1_conc_in

t = mpc_params["t_step"] * np.arange(n_steps)
plt.figure(figsize=(7, 3))
plt.step(t, dist_inputs[:, 0], where="post", label="tank_1_v_dot_in")
plt.step(t, dist_inputs[:, 1], where="post", label="tank_1_conc_in")
plt.grid()
plt.ylim([0, None])
plt.title("Disturbance Inputs")
plt.xlabel("Time [hours]")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# ========================================
# 7. Run Closed-Loop Simulation
# ========================================

print("\nRunning closed-loop simulation...")
print(f"Simulation steps: {n_steps}")
print(f"Time step: {mpc_params['t_step']} hours")

x0 = x0_init
for k in range(n_steps):
    # Get control action from MPC
    u0 = mpc.make_step(x0)

    # Set disturbance inputs in simulator
    simulator.x0["tank_1_v_dot_in"] = dist_inputs[k, 0]
    simulator.x0["tank_1_conc_in"] = dist_inputs[k, 1]

    # Simulate system
    y_next = simulator.make_step(u0)

    # For perfect controller testing, use true state from simulator
    x0 = simulator.x0

    if k % 10 == 0:
        print(f"Step {k}/{n_steps}")

print("Simulation complete!")

In [ ]:
# Put simulation results into a Pandas DataFrame

# Simulator.make_step stores y(k+1) in the '_y' field, not y(k).
# Therefore shift the values back forward one time step and insert
# the initial condition at t = 0.
data_y = simulator.data["_y"].copy()
data_y = np.roll(data_y, 1, axis=0)
data_y[0, :] = np.array(y0_init).flatten()

sim_results = pd.concat(
    {
        "time": pd.DataFrame(simulator.data["_time"]),
        "manipulated_inputs": pd.DataFrame(
            simulator.data["_u"],
            columns=pd.Index(simulator.model._u.keys()).drop("default"),
        ),
        "states": pd.DataFrame(
            simulator.data["_x"], columns=pd.Index(simulator.model._x.keys())
        ),
        "outputs": pd.DataFrame(
            data_y, columns=pd.Index(simulator.model._y.keys()).drop("default")
        ),
    },
    axis=1,
)

# Save results to CSV file
sim_results.to_csv(results_dir / "feed_conc_ctrl_sim_results.csv", index=False)

sim_results.info()

In [ ]:
# ========================================
# 8. Visualize Results
# ========================================

# Define units
flow_units = r"$m^3/h$"
conc_units = r"$tons/m^3$"
level_units = r"$m$"
ratio_units = r"(fraction)"

units = {
    "tank_1_v_dot_in": flow_units,
    "tank_2_v_dot_in": flow_units,
    "mixer_v_dot_out": flow_units,
    "tank_1_conc_in": conc_units,
    "tank_2_conc_in": conc_units,
    "tank_1_conc_out": conc_units,
    "tank_2_conc_out": conc_units,
    "mixer_conc_out": conc_units,
    "tank_1_L": level_units,
    "tank_2_L": level_units,
    "mixer_r_1": ratio_units,
}

# Define plot structure
plot_info = {
    "Tank Levels": {
        "Tank 1": "tank_1_L",
        "Tank 2": "tank_2_L",
        "Tank 3": "tank_3_L",
        "Tank 4": "tank_4_L",
    },
    "Flow Rates": {
        "Tank 1 In": "tank_1_v_dot_in",
        "Tank 2 In": "tank_2_v_dot_in",
        "Mixer In 1": "mixer_v_dot_in_1",
        "Mixer In 2": "mixer_v_dot_in_2",
        "Tank 4 Out": "tank_4_v_dot_out",
    },
    "Concentrations": {
        "Feed": "tank_1_conc_in",
        "Tank 1": "tank_1_conc_out",
        "Tank 2": "tank_2_conc_out",
        "Tank 3": "tank_3_conc_out",
        "Tank 4": "tank_4_conc_out",
    },
}

# Prepare data: drop multi-level column index and remove duplicates
var_data = sim_results.droplevel(0, axis=1)
var_data = var_data.loc[:, ~var_data.columns.duplicated()]

# Create plots
fig, axes = make_tsplots(
    var_data, plot_info, units=units, legend_loc="outside right"
)

# Add setpoint lines to level and concentration subplots
for ax, (title, sub_plot_info) in zip(axes, plot_info.items()):
    if title in ["Tank Levels", "Concentrations"]:
        for var_name in sub_plot_info.values():
            if var_name in setpoints:
                ax.axhline(
                    y=setpoints[var_name],
                    color="gray",
                    linestyle="--",
                    alpha=0.5,
                    linewidth=1,
                )

plt.tight_layout()
filename = "mpc_demo_2_simulation.png"
plt.savefig(plot_dir / filename)
plt.close()

In [ ]:
# ========================================
# Performance Summary
# ========================================
print("\n=== Performance Summary ===")

# Get final values from outputs and manipulated inputs
y_final = sim_results["outputs"].iloc[-1]
u_final = sim_results["manipulated_inputs"].iloc[-1]

# Combine MVs and CVs into a single dict for easy lookup
mv_names = control_design["manipulated_variables"]
cv_names = control_design["controlled_variables"]

for var_name, sp in setpoints.items():
    # Check if variable is in outputs (CV) or manipulated_inputs (MV)
    if var_name in y_final.index:
        final_value = y_final[var_name]
    elif var_name in u_final.index:
        final_value = u_final[var_name]
    else:
        print(f"Warning: {var_name} not found in outputs or inputs")
        continue
    
    print(f"Final {var_name}: {final_value:.3f} (target: {sp})")


# ========================================
# 9. Print Performance Metrics
# ========================================

# Extract time and data from sim_results for all setpoint variables
time = sim_results["time"][0].values

# Build combined data dict for all variables with setpoints (CVs and MVs)
setpoint_data_dict = {}
for var_name in setpoints.keys():
    if var_name in sim_results["outputs"].columns:
        setpoint_data_dict[var_name] = sim_results["outputs"][var_name].values
    elif var_name in sim_results["manipulated_inputs"].columns:
        setpoint_data_dict[var_name] = sim_results["manipulated_inputs"][var_name].values
    else:
        print(f"Warning: {var_name} not found in simulation results")

# Calculate tracking errors
tracking_errors = {}
for var_name, sp in setpoints.items():
    if var_name not in setpoint_data_dict:
        continue
    
    var_data = setpoint_data_dict[var_name]
    
    # RMS error
    rms_error = np.sqrt(np.mean((var_data - sp) ** 2))
    tracking_errors[var_name] = rms_error

print("\n=== Performance Metrics ===")
for var_name, error in tracking_errors.items():
    print(f"{var_name}: RMS error = {error:.4f}")

# Settling time (time to reach within 5% of setpoint)
print("\n=== Settling Times ===")
for var_name, sp in setpoints.items():
    if var_name not in setpoint_data_dict:
        continue
        
    var_data = setpoint_data_dict[var_name]
    tolerance = 0.05 * abs(sp)
    
    # Find first time within tolerance and stays within
    within_tolerance = np.abs(var_data - sp) < tolerance
    if np.any(within_tolerance):
        settling_idx = np.argmax(within_tolerance)
        settling_time = settling_idx * mpc_params["t_step"]
        print(f"{var_name}: {settling_time:.1f} hours")
    else:
        print(f"{var_name}: Not settled within {tolerance:.3f}")

# Additional metrics
print("\n=== Final Values ===")
for var_name, sp in setpoints.items():
    if var_name not in setpoint_data_dict:
        continue
        
    var_data = setpoint_data_dict[var_name]
    final_value = var_data[-1]
    final_error = final_value - sp
    print(
        f"{var_name}: {final_value:.3f} (target: {sp:.1f}, error: {final_error:+.3f})"
    )

# Control effort
print("\n=== Control Effort ===")
mv_data_dict = {}
for mv_name in control_design["manipulated_variables"]:
    mv_data_dict[mv_name] = sim_results["manipulated_inputs"][mv_name].values

for mv_name in control_design["manipulated_variables"]:
    mv_data = mv_data_dict[mv_name]
    print(f"{mv_name}: mean={np.mean(mv_data):.3f}, std={np.std(mv_data):.3f}")

# Total variation (measure of control smoothness)
print("\n=== Control Smoothness (Total Variation) ===")
for mv_name in control_design["manipulated_variables"]:
    mv_data = mv_data_dict[mv_name]
    tv = np.sum(np.abs(np.diff(mv_data)))
    print(f"{mv_name} TV: {tv:.3f}")

## Define State Estimator - Extended Kalman Filter (EKF)

In [ ]:
# Setup Estimator
estimator_design = {
    "meas_noise_std": {
        "tank_1_L": 0.1,
        "tank_1_conc_out": 0.5,
        "tank_2_L": 0.1,
        "tank_2_conc_out": 0.5,
        "mixer_conc_out": 0.5,
    },
    "process_noise_std": {
        "tank_1_L": 0.01,
        "tank_1_m": 0.01,
        "tank_2_L": 0.01,
        "tank_2_m": 0.01,
        "tank_1_conc_in": 1.0,
        "tank_2_conc_in": 1.0,
        "mixer_v_dot_out": 1.0,
    },
    "initial_state_covariance": {
        "tank_1_L": 1000.0,
        "tank_1_m": 1000.0,
        "tank_2_L": 1000.0,
        "tank_2_m": 1000.0,
        "tank_1_conc_in": 1000.0,
        "tank_2_conc_in": 1000.0,
        "mixer_v_dot_out": 1000.0,
    },
    "kwargs": {
        "t_step": 1.0,  # Time step (hours)
    },
}

# ========================================
# Create EKF estimator
# ========================================
estimator = do_mpc.estimator.EKF(model)

# ========================================
# Build measurement noise covariance matrix (R)
# ========================================
# R is a diagonal matrix with measurement noise variances
R = np.diag(
    [
        estimator_design["meas_noise_std"][name] ** 2
        for name in control_design["controlled_variables"]
    ]
)
print(f"Measurement noise covariance (R) shape: {R.shape}")

# ========================================
# Build process noise covariance matrix (Q)
# ========================================
# Q is a diagonal matrix with process noise variances
n_x = model.n_x
Q = np.diag(
    [
        estimator_design["process_noise_std"][name] ** 2
        for name in model.x.keys()
    ]
)
print(f"Process noise covariance (Q) shape: {Q.shape}")

# ========================================
# Build initial state covariance matrix (P0)
# ========================================
# P0 represents uncertainty in initial state estimate
P0 = np.diag(
    [
        estimator_design["initial_state_covariance"][name]
        for name in model.x.keys()
    ]
)
print(f"Initial covariance (P0) shape: {P0.shape}")

# ========================================
# Setup the EKF with covariances
# ========================================
estimator.settings.t_step = estimator_design["kwargs"]["t_step"]
estimator.settings.P_x = P0  # Initial state covariance
estimator.settings.P_v = Q  # Process noise covariance
estimator.settings.P_w = R  # Measurement noise covariance

# Setup the estimator
estimator.setup()

# Set initial state estimate
x0 = np.zeros((model.n_x, 1))

estimator.x0 = x0
estimator.reset_history()

print("\nEKF setup complete.")
print(f"Time step: {estimator_design['kwargs']['t_step']} hours")
print(f"Number of states: {estimator.model.n_x}")
print(f"Number of known inputs: {estimator.model.n_u}")
print(f"Number of measurements: {estimator.model.n_y}")


## Generate Simulated Input-Output Data to Test Estimator

In [ ]:
dt = 1.0
system_dt = StateSpaceModelDTFromCT(system, dt=dt)
system_dt

nT = 100
simulate = make_n_step_simulation_function_from_model(system_dt, nT)
simulate

In [ ]:
# Example: Run estimator in a loop
for k in range(100):
    # Get measurements (from real system or simulator)
    y_measured = np.random.randn(n_y, 1) * 0.1 + 1.0  # Example

    # Get control inputs
    u_applied = np.array([[0.5], [0.5], [0.6]])  # Example

    # Run EKF to estimate states (including disturbances)
    x_estimated = estimator.make_step(y_measured)

    # Extract disturbance estimates
    dist_estimates = {}
    for i, dist_name in enumerate(d.keys()):
        state_idx = len(x) + i  # Disturbances come after system states
        dist_estimates[dist_name] = float(x_estimated[state_idx])

    print(f"Step {k}: Disturbances = {dist_estimates}")